In [1]:
# Adjustable Privacy - train_on_obfuscated.ipynb
# - Train a machine (srtong adversary or utilizer) on obfuscated (image) dataset to infer a specific feature.
# - Uses obfuscated image dataset (CelebA).
# - Saves models after each epoch number (to google drive and locally).
# - It can stop and resume training.
# - Draws loss and accuracy plots and saves them (to google drive).
# - For adversary test on obfuscated testset, and for utilizer test on original dataset.
# - Also it can load models and draw plots (from google drive).
# - Also it loads the weak adversary and evaluate it on obfuscated testset and reports its accuracy.
# - It shows some obfuscated images and saves them (to google drive)
# - You can manage notebook parameters in parser block

In [2]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
import torchvision.utils as vutils
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import random
import shutil
from zipfile import ZipFile
import os
from tqdm import tqdm
from collections import OrderedDict
import time
from math import floor
import argparse

In [55]:
# Parser
parser = argparse.ArgumentParser(description='Adjustable Privacy - Train a machine (srtong adversary or utilizer) on obfuscated (image) dataset to infer a specific feature. '
                                 + 'Uses obfuscated image dataset (CelebA). '
                                 + 'Saves models after each epoch number (to google drive and locally). '
                                 + 'It can stop and resume training. '
                                 + 'Draws loss and accuracy plots and saves them (to google drive). '
                                 + 'For adversary test on obfuscated testset, and for utilizer test on original dataset. '
                                 + 'Also it can load models and draw plots (from google drive).'
                                 + 'Also it loads the weak adversary and evaluate it on obfuscated testset and reports its accuracy. '
                                 + 'It shows some obfuscated images and saves them (to google drive)')

parser.add_argument('--resume', default = False, help = 'Accepts "True" or "False". ')
parser.add_argument('--last_epoch', type=int, default = 0, help = 'In case of resumming training use last saved epoch number and in case of loading a model, set to model number.')
parser.add_argument('--utilizer_target_index', type=int, required=True, help = '(only for utilizer) gender(20), smiling(31), open mouth(21), and high chickbone(19)')
parser.add_argument('--adversary_or_utilizer', type=str, default = 'utilizer', help = 'This model should train on obfuscated dataset as "utilizer" or "adversary"')
parser.add_argument('--save_path', type=str, required=True, help = 'Full path on your google drive to save model and plots. And also load from it. Like "drive/MyDrive/adjustable-privacy/Models/adv-or-utl/utl-GS-model0-g3000-f20/"')
parser.add_argument('--epoch_numbers', type=int, default = 20, help = 'Number of epochs to train model. (when you want load a model, it should set to that model number)')
parser.add_argument('--download_dataset', default = False, help = 'Accepts "True" or "False". Download CelebA dataset or use CelebA shared directory.')
parser.add_argument('--dataset_path', type=str, default = "", help = '(if --download_dataset=False) Full path on your google drive to CelebA shared directory shortcut. Like "drive/MyDrive/adjustable-privacy/Datasets/CelebA/"')
parser.add_argument('--pic_name', type=str, default = "obf_test_pics.png", help = 'File name of saved obfuscated first 8 pics of test set. Like "obf_test_pics.png"')
parser.add_argument('--obfuscated_dataset_version_name', type=str, default = "", help = 'a short name specify obfuscated dataset like:"GS-model0-g3000-f20" ')
parser.add_argument('--obfuscated_dataset_path', type=str, required=True, help = 'Full path on your google drive to load obfuscated dataset from. Like "drive/MyDrive/adjustable-privacy/Datasets/ObfuscatedCelebA/"')
parser.add_argument('--test_weak_adversary', default = False, help = 'Accepts "True" or "False". ')
parser.add_argument('--weak_adversary_model_path', type=str, required=True, help = 'Full path on your google drive to load weak adversary model from. Like "drive/MyDrive/adjustable-privacy/Models/CelebA-G/"')
parser.add_argument('--weak_adversary_model_number', type=int, required=True, help = 'Weak Adversary model number')

command_string = "--resume False" \
" --last_epoch 0" \
" --utilizer_target_index 20" \
" --adversary_or_utilizer utilizer" \
" --save_path drive/MyDrive/adjustable-privacy/Models/adv-or-utl/utl-GS-model0-g3000-f20/" \
" --epoch_numbers 2" \
" --download_dataset False" \
" --dataset_path drive/MyDrive/adjustable-privacy/Datasets/CelebA/" \
" --pic_name obf_test_pics.png" \
" --obfuscated_dataset_version_name GS-model0-g3000-f20" \
" --obfuscated_dataset_path drive/MyDrive/adjustable-privacy/Datasets/ObfuscatedCelebA/" \
" --test_weak_adversary True" \
" --weak_adversary_model_path drive/MyDrive/adjustable-privacy/Models/CelebA-G/" \
" --weak_adversary_model_number 2"

args = parser.parse_args(command_string.split())

In [56]:
# Hyper parameters:
isFirstRun = args.resume=='False'
lastRunEpochNumber = args.last_epoch
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.0
celeba_male_index = 20
celeba_smiling_index = 31
celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19

if args.adversary_or_utilizer=='utilizer':
  is_adv = False
else:
  if args.adversary_or_utilizer=='adversary':
    is_adv = True

using_index = args.utilizer_target_index
adv_using_index = celeba_male_index
if is_adv:
  using_index = adv_using_index

saving_pic_file_name = args.pic_name

learning_rate = 0.001 #0.001
batch_size = 64

download_dataset = args.download_dataset=='True'
dataset_folder_path = args.dataset_path

data_dir = 'celeba'
saving_path = args.save_path

# modified dataset:
m_version = args.obfuscated_dataset_version_name
on_drive_zipped_file_path = args.obfuscated_dataset_path + m_version + '/celeba.zip'
modified_dataset_saving_path = 'modifiedDatasets/' + m_version + '/'
copied_and_unzipped_from_drive = False
load_main_dataset = True
suffling_main_train_data = True
suffling_modified_train_data = True

# Adversary
load_adversary = args.test_weak_adversary=='True'
epoch_number_of_adversary_model = args.weak_adversary_model_number
adversary_saving_path = args.weak_adversary_model_path

# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Number of training epochs
num_epochs = args.epoch_numbers

In [ ]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# getting dataset ready from shared directory
if not download_dataset:
    dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
    list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
    identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
    list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
    list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
    list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)

In [ ]:
# make saving path dir
try:
    os.makedirs(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

In [ ]:
# copy and unzip from drive to this env:
m_celeba_dir = modified_dataset_saving_path + 'celeba/'
img_celeba_dir = modified_dataset_saving_path + 'celeba/img_align_celeba/'

try:
  os.makedirs(m_celeba_dir)
  print("modified data folder created successfully")
except OSError as e:
  print("Error: one of %s" % (e.strerror))

if not copied_and_unzipped_from_drive:
  shutil.copyfile(on_drive_zipped_file_path, modified_dataset_saving_path + 'celeba.zip')
  archive1 = modified_dataset_saving_path + 'celeba.zip'
  with ZipFile(archive1, 'r') as zip:
      zip.extractall(m_celeba_dir)

In [32]:
# Define transforms
train_transforms = transforms.Compose([transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [33]:
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed))
  return used_dataset

In [34]:
# Load Datas
if load_main_dataset:
    train_set = datasets.CelebA(root='', download=download_dataset, split='train', target_type=["attr", "identity"], transform=train_transforms)
    test_set = datasets.CelebA(root='', download=download_dataset, split='test', target_type=["attr", "identity"], transform=test_transforms)
    valid_set = datasets.CelebA(root='', download=download_dataset, split='valid', target_type=["attr", "identity"], transform=test_transforms)

    # shorten Dataset
    if not use_whole_dataset:
      train_set = shorten_dataset(train_set, usage_percent)

    # DataLoader
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=suffling_main_train_data, num_workers=workers, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers, drop_last=True)
    valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)


In [35]:
# load modified dataset:
m_transforms = transforms.Compose([transforms.ToTensor()])
m_train_set = datasets.CelebA(root=modified_dataset_saving_path, download=False, split='train', target_type=["attr", "identity"], transform=m_transforms)
m_valid_set = datasets.CelebA(root=modified_dataset_saving_path, download=False, split='valid', target_type=["attr", "identity"], transform=m_transforms)
m_test_set = datasets.CelebA(root=modified_dataset_saving_path, download=False, split='test', target_type=["attr", "identity"], transform=m_transforms)

# shorten Dataset
if not use_whole_dataset:
  m_train_set = shorten_dataset(m_train_set, usage_percent)

m_train_loader = torch.utils.data.DataLoader(m_train_set, batch_size=batch_size, shuffle=suffling_modified_train_data, num_workers=workers, drop_last=True)
m_valid_loader = torch.utils.data.DataLoader(m_valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)
m_test_loader = torch.utils.data.DataLoader(m_test_set, batch_size=batch_size, num_workers=workers, drop_last=True)

In [36]:
# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

In [37]:
# custom weights initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [38]:
# Utilizer or Adversary Model
class UtlAdvModel(nn.Module):
    def __init__(self, ngpu):
        super(UtlAdvModel, self).__init__()
        self.ngpu = ngpu

        # input is nc x 64 x 64
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # classifier: 
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # classifier:
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1

In [ ]:
# Create the UTLAdv
utilizerAdversaryModel = UtlAdvModel(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    utilizerAdversaryModel = nn.DataParallel(utilizerAdversaryModel, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
utilizerAdversaryModel.apply(weights_init)

In [ ]:
# total parameters
total_params = sum(p.numel() for p in utilizerAdversaryModel.parameters())
print(f"{total_params:,} total parameters.")

In [41]:
utilizerAdversaryCriterion = nn.NLLLoss()
utilizerAdversaryOptimizer = optim.Adam(utilizerAdversaryModel.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [42]:
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res,
                'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [43]:
# Function - Load:
def load_model(name, number, model, device):

  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [44]:
# Save Start Checkpoint
if(isFirstRun):
  utl_adv_res = {'train_losses': [],
             'valid_losses': [],
             'test_y1_acc': [],
             'epoch_number': 0,
           };
  save_model('ins', 0, utilizerAdversaryModel, utl_adv_res)

In [45]:
# Load Last Checkpoint:
utl_adv_load = load_model('ins', lastRunEpochNumber, utilizerAdversaryModel, device)

train_losses = utl_adv_load['res']['train_losses']
valid_losses = utl_adv_load['res']['valid_losses']
test_y1_acc = utl_adv_load['res']['test_y1_acc']
last_epoch = utl_adv_load['res']['epoch_number']

In [46]:
# Function - training function
def fit(model, train_loader, optimizer, criterion):
    print('Training')
    model.train()

    train_loss = 0.0
    prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in prog_bar:
        inputs, labels = data[0], data[1]
        inputs = inputs.to(device)
        main_target = labels[0][:, using_index]
        main_target = main_target.to(device)
        model.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, main_target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)
    return train_loss

In [47]:
# Function - validation function
def validate(model, valid_loader, criterion):
    print('Validating')
    model.eval()
    valid_loss = 0.0

    prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = labels[0][:, using_index]
            main_target = main_target.to(device)
            outputs = model.forward(inputs)
            loss = criterion(outputs, main_target)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(valid_loader)
        return valid_loss

In [48]:
# Calc Accuracy
def calcAccuracyTest(model, test_loader):
    print('Testing')
    model.to(device)
    print("Calculating Accuracy...")
    model.eval()
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = labels[0][:, using_index]
            main_target = main_target.to(device)
            output = model(inputs)
            ps_y1 = torch.exp(output)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            equals_y1 = top_class_y1 == main_target.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            y1_accuracy += (acc_y1 / len(equals_y1))
    y1_accuracy = y1_accuracy / len(test_loader)
    return y1_accuracy

In [ ]:
# Training Loop
utilizerAdversaryModel.to(device)
save_every_epoch = 1

if is_adv:
  here_test_loader = m_test_loader
else:
  here_test_loader = test_loader

start = time.time()
print("Starting Training Loop...")

for epoch in range(last_epoch+1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}: ")

    train_loss = fit(utilizerAdversaryModel, m_train_loader, utilizerAdversaryOptimizer, utilizerAdversaryCriterion)
    valid_loss = validate(utilizerAdversaryModel, m_valid_loader, utilizerAdversaryCriterion)
    y1_accuracy = calcAccuracyTest(utilizerAdversaryModel, here_test_loader)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    test_y1_acc.append(y1_accuracy)

    utl_adv_res = {'train_losses': train_losses,
               'valid_losses': valid_losses,
               'test_y1_acc': test_y1_acc,
               'epoch_number': epoch
                }
    if epoch % save_every_epoch == 0:
      save_model('ins', epoch, utilizerAdversaryModel, utl_adv_res)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}")
    print(f"Male Accuracy on unobfuscated Testset: {y1_accuracy:.4f}")

end = time.time()
print(f"Training time: {(end-start)/60:.3f} minutes")

print('TRAINING COMPLETE')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print('Loss plot...')

# loss plots
plt.figure(figsize=(10,7))
plt.title("Train-Valid Loss Trend on obfuscated data")
plt.plot(train_losses, color='green', label='Training Loss')
plt.plot(valid_losses, color='blue', label='Validation Loss')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.savefig(saving_path + "loss_plot.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.title("S Accuracy Trend")
plt.plot(test_y1_acc, color='green', label='S Test set Accuracy (on unobfuscated data)')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.savefig(saving_path + "accuracy_test_plot.png")
plt.show()

In [52]:
# For Weak Adversary
# Adversary Model
class AdvModel(nn.Module):
    def __init__(self, ngpu):
        super(AdvModel, self).__init__()
        self.ngpu = ngpu
        
        # input is nc x 64 x 64 
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # classifier
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # classifier:
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1

In [53]:
# For Weak Adversary
def adv_load_model(name, number, model, device):
  
  checkpoint = torch.load(adversary_saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [57]:
# For Weak Adversary
# Create the ADV
adversaryModel = AdvModel(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    adversaryModel = nn.DataParallel(adversaryModel, list(range(ngpu)))

adv_load = adv_load_model('ins', epoch_number_of_adversary_model, adversaryModel, device)

In [58]:
# For Weak Adversary
# Calc Accuracy
def calcAdvAccuracyTest(model, test_loader):
    print('Testing')
    model.to(device)
    print("Calculating Accuracy...")
    model.eval()
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            main_target = labels[0][:, adv_using_index]
            main_target = main_target.to(device)
            output = model(inputs)
            ps_y1 = torch.exp(output)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            equals_y1 = top_class_y1 == main_target.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            y1_accuracy += (acc_y1 / len(equals_y1))            
    y1_accuracy = y1_accuracy / len(test_loader)
    return y1_accuracy

In [ ]:
# For Weak Adversary
if load_adversary:
  # Test on obfuscated data
  adversaryModel.to(device)
  weak_accuracy = calcAdvAccuracyTest(adversaryModel, m_test_loader)
  print(f"\n Weak Adversary Accuracy on Testset: {weak_accuracy:.6f}")

In [ ]:
first_batch1 = next(iter(test_loader))
first_batch2 = next(iter(m_test_loader))

first_batch1[0][0] = first_batch1[0][5]
first_batch1[0][1] = first_batch1[0][16]
first_batch1[0][2] = first_batch1[0][18]
first_batch1[0][3] = first_batch1[0][21]
first_batch1[0][4] = first_batch1[0][29]
first_batch1[0][5] = first_batch1[0][43]
first_batch1[0][6] = first_batch1[0][44]
first_batch1[0][7] = first_batch1[0][54]

first_batch2[0][0] = first_batch2[0][5]
first_batch2[0][1] = first_batch2[0][16]
first_batch2[0][2] = first_batch2[0][18]
first_batch2[0][3] = first_batch2[0][21]
first_batch2[0][4] = first_batch2[0][29]
first_batch2[0][5] = first_batch2[0][43]
first_batch2[0][6] = first_batch2[0][44]
first_batch2[0][7] = first_batch2[0][54]

plt.figure(figsize=(16,16))
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(first_batch1[0][0:8].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.figure(figsize=(16,16))
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(first_batch2[0][0:8].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.savefig(saving_path + saving_pic_file_name)